In [166]:
import pandas as pd
import numpy as np
from datetime import datetime
import scipy
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
import random

In [167]:
promotion=pd.read_csv(r"C:\\Users\\mhamu\\DScience_Py3\\assignment\\Promotion_analysis\\SAS_Codes\\data\\1_promotion.csv",sep=';' ,encoding='latin-1',low_memory=False)
trx=pd.read_csv(r"C:\\Users\\mhamu\\DScience_Py3\\assignment\\Promotion_analysis\\SAS_Codes\\data\\2_trx.csv",sep=';' ,encoding='latin-1',low_memory=False)
promotion.info()
trx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169 entries, 0 to 1168
Data columns (total 6 columns):
dateFrom         1169 non-null object
dateThru         1169 non-null object
Group            1169 non-null object
promotionType    1169 non-null object
sku              1169 non-null int64
store            31 non-null object
dtypes: int64(1), object(5)
memory usage: 54.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357772 entries, 0 to 357771
Data columns (total 7 columns):
store     357772 non-null object
sku       357772 non-null int64
sale1     357772 non-null int64
sale2     357772 non-null int64
sale3     357772 non-null int64
return    357772 non-null int64
date      357772 non-null object
dtypes: int64(5), object(2)
memory usage: 19.1+ MB


In [125]:
def convertdate(raw,split):
    data=raw.str.split(split,expand=True)
    data.columns=['D','M','Y']
    data['DATE']=pd.to_datetime(data['M']+'.'+data['D']+'.'+data['Y'])
    return data['DATE']

In [126]:
#convert text DD/MM/YYYY to MM/DD/YYYY date format
promotion['dateFrom']=convertdate(promotion['dateFrom'],'.')
promotion['dateThru']=convertdate(promotion['dateThru'],'.')
trx['date']=convertdate(trx['date'],'.')
#t=[promotion['dateFrom'],promotion['dateThru'],trx['date']]

In [127]:
print(promotion['dateFrom'].max())

2014-12-05 00:00:00


In [128]:
#print(promotion.head(n=3))
#print(trx.head(n=3))
promotion['store']=promotion['store'].fillna('GENERAL')
promotion['STORE_ALT']=promotion['store']
#promotion['STORE_ALT'].unique()

#convert store types
trx.loc[trx['store'].isin(['STORE9','STORE12']),'STORE_ALT']=trx['store']
trx['STORE_ALT']=trx['STORE_ALT'].fillna('GENERAL')

#convert sku to text
promotion['SKU_ALT']=promotion['sku'].apply(str)
trx['SKU_ALT']=trx['sku'].apply(str)


promotion['KEY'] = promotion[['STORE_ALT', 'SKU_ALT']].apply(lambda x: ''.join(x), axis=1)
trx['KEY'] = trx[['STORE_ALT', 'SKU_ALT']].apply(lambda x: ''.join(x), axis=1)

In [129]:
#trx.pivot_table(values=['sale1'],index=['store','STORE_ALT'],aggfunc=lambda x: sum(x))
#promotion.pivot_table(values=['dateFrom'],index=['store','STORE_ALT'],aggfunc=lambda x: max(x))
#print(promotion['STORE_ALT'].unique())
#print(promotion['store'].unique())
#print(trx['STORE_ALT'].unique())
#print(trx['store'].unique())


In [130]:
promotion.head()

,dateFrom,dateThru,Group,promotionType,sku,store,STORE_ALT,SKU_ALT,KEY
0,2012-01-01,2012-01-31,Special,Media,1122735,GENERAL,GENERAL,1122735,GENERAL1122735
1,2012-01-01,2012-01-31,Special,Media,1114952,GENERAL,GENERAL,1114952,GENERAL1114952
2,2012-01-01,2012-01-31,Special,Media,1100480,GENERAL,GENERAL,1100480,GENERAL1100480
3,2012-01-06,2012-02-02,flyer,Flyer,2014630,GENERAL,GENERAL,2014630,GENERAL2014630
4,2012-01-06,2012-02-02,flyer,Flyer,1111699,GENERAL,GENERAL,1111699,GENERAL1111699


In [131]:
trx.head()

,store,sku,sale1,sale2,sale3,return,date,STORE_ALT,SKU_ALT,KEY
0,STORE4,1084123,0,0,0,-6,2012-04-01,GENERAL,1084123,GENERAL1084123
1,STORE12,1116257,0,0,0,-1,2012-04-01,STORE12,1116257,STORE121116257
2,STORE7,1116407,0,0,0,-1,2012-04-01,GENERAL,1116407,GENERAL1116407
3,STORE10,1117146,0,1,0,0,2012-04-01,GENERAL,1117146,GENERAL1117146
4,STORE7,1117427,0,0,0,-1,2012-04-01,GENERAL,1117427,GENERAL1117427


In [132]:
#merge and flag promoted
trx_prom=pd.merge(trx,promotion,on='KEY',how='left')
trx_prom.loc[(trx_prom['dateThru']>=trx_prom['date']) & (trx_prom['date']>=trx_prom['dateFrom']),'PROMOTED']=1
trx_prom['PROMOTED']=trx_prom['PROMOTED'].fillna(0)

In [133]:
trx_prom['promotionType'].replace(to_replace=["Flyer"],value='FLYER', inplace=True)
trx_prom['promotionType'].replace(to_replace=["Mother's Day"],value='MTHRSDY', inplace=True)
trx_prom['promotionType'].replace(to_replace=["Arrival"],value='ARRIVAL', inplace=True)
trx_prom['promotionType'].replace(to_replace=["\x1andir"],value='INDIR', inplace=True)
trx_prom['promotionType'].replace(to_replace=["Don\x92t Miss"],value='DONTMISS', inplace=True)
trx_prom['promotionType'].replace(to_replace=["Dept extra"],value='DEPTEXTR', inplace=True)
trx_prom['promotionType'].replace(to_replace=["VACATION OUTDOOR MAY"],value='VACOUTDOOR', inplace=True)
trx_prom['promotionType'].replace(to_replace=["PROMO PRODUCT"],value='PROMOPROD', inplace=True)
trx_prom['promotionType'].replace(to_replace=["Media"],value='MEDIA', inplace=True)
trx_prom['promotionType'].replace(to_replace=["Media2"],value='MEDIA2', inplace=True)
trx_prom['promotionType']=trx_prom['promotionType'].fillna('NONE')

trx_prom['Group'].replace(to_replace=["flyer"],value='FLYER', inplace=True)
trx_prom['Group'].replace(to_replace=["Special"],value='SPECIAL', inplace=True)
trx_prom['Group'].replace(to_replace=["Other"],value='OTHER', inplace=True)
trx_prom['Group'].replace(to_replace=["Money"],value='MONEY', inplace=True)
trx_prom['Group']=trx_prom['Group'].fillna('NONE')

trx_prom['GroupPrm'] = trx_prom[['Group', 'promotionType']].apply(lambda x: '_'.join(x), axis=1)


In [134]:
print(trx_prom['Group'].unique())
print(trx_prom['promotionType'].unique())
print(trx_prom['GroupPrm'].unique())

['FLYER' 'NONE' 'SPECIAL' 'OTHER' 'MONEY' 'STORE']
['FLYER' 'NONE' 'MTHRSDY' 'ARRIVAL' 'INDIR' 'DONTMISS' 'DEPTEXTR' 'ERP'
 'MEDIA' 'VACOUTDOOR' 'PROMOPROD' 'MEDIA2']
['FLYER_FLYER' 'NONE_NONE' 'SPECIAL_MTHRSDY' 'SPECIAL_ARRIVAL'
 'SPECIAL_INDIR' 'SPECIAL_DONTMISS' 'OTHER_DEPTEXTR' 'MONEY_ERP'
 'SPECIAL_MEDIA' 'STORE_VACOUTDOOR' 'STORE_PROMOPROD' 'SPECIAL_MEDIA2']


In [135]:
trx_prom['TOT_SALES']=trx_prom[['sale1','sale2','sale3']].apply(lambda x: sum(x),axis=1)
trx_prom['NET_SALES']=trx_prom[['TOT_SALES','return']].apply(lambda x: sum(x),axis=1)



In [136]:
trx_prom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1239901 entries, 0 to 1239900
Data columns (total 22 columns):
store_x          1239901 non-null object
sku_x            1239901 non-null int64
sale1            1239901 non-null int64
sale2            1239901 non-null int64
sale3            1239901 non-null int64
return           1239901 non-null int64
date             1239901 non-null datetime64[ns]
STORE_ALT_x      1239901 non-null object
SKU_ALT_x        1239901 non-null object
KEY              1239901 non-null object
dateFrom         1187309 non-null datetime64[ns]
dateThru         1187309 non-null datetime64[ns]
Group            1239901 non-null object
promotionType    1239901 non-null object
sku_y            1187309 non-null float64
store_y          1187309 non-null object
STORE_ALT_y      1187309 non-null object
SKU_ALT_y        1187309 non-null object
PROMOTED         1239901 non-null float64
GroupPrm         1239901 non-null object
TOT_SALES        1239901 non-null int64
NET_SA

In [137]:
trx_prom_grp=trx_prom.pivot_table(values=['PROMOTED'],index=['KEY','date'],columns=['GroupPrm'],aggfunc=lambda x: max(x),fill_value=0)

In [138]:
trx_prom_grp.head()

PROMOTED                                     \
GroupPrm                  FLYER_FLYER MONEY_ERP NONE_NONE OTHER_DEPTEXTR   
KEY            date                                                        
GENERAL1016030 2012-05-13           0         0         0              0   
               2012-05-29           0         0         0              0   
               2012-06-04           0         0         0              0   
               2012-06-24           0         0         0              0   
               2012-06-25           0         0         0              0   

                                                                          \
GroupPrm                  SPECIAL_ARRIVAL SPECIAL_DONTMISS SPECIAL_INDIR   
KEY            date                                                        
GENERAL1016030 2012-05-13               0                0             0   
               2012-05-29               0                0             0   
               2012-06-04               0                0             0   
               2012-06-24               0                0             0   
               2012-06-25               0                0             0   

                                                                        \
GroupPrm                  SPECIAL_MEDIA SPECIAL_MEDIA2 SPECIAL_MTHRSDY   
KEY            date                                                      
GENERAL1016030 2012-05-13             0              0               0   
               2012-05-29             0              0               0   
               2012-06-04             0              0               0   
               2012-06-24             0              0               0   
               2012-06-25             0              0               0   

                                                            
GroupPrm                  STORE_PROMOPROD STORE_VACOUTDOOR  
KEY            date                                         
GENERAL1016030 2012-05-13               0                0  
               2012-05-29               0                0  
               2012-06-04               0                0  
               2012-06-24               0                0  
               2012-06-25               0                0

In [139]:
trx_prom_grp_table = pd.DataFrame(trx_prom_grp.to_records())
trx_prom_grp_table.columns = [hdr.replace("('PROMOTED', '", "CAMP_").replace("')", "") 
                              for hdr in trx_prom_grp_table.columns]
trx_prom_grp_table.columns

Index(['KEY', 'date', 'CAMP_FLYER_FLYER', 'CAMP_MONEY_ERP', 'CAMP_NONE_NONE',
       'CAMP_OTHER_DEPTEXTR', 'CAMP_SPECIAL_ARRIVAL', 'CAMP_SPECIAL_DONTMISS',
       'CAMP_SPECIAL_INDIR', 'CAMP_SPECIAL_MEDIA', 'CAMP_SPECIAL_MEDIA2',
       'CAMP_SPECIAL_MTHRSDY', 'CAMP_STORE_PROMOPROD',
       'CAMP_STORE_VACOUTDOOR'],
      dtype='object')

In [140]:
trx_w_grp_prom=pd.merge(trx,trx_prom_grp_table,on=['KEY','date'],how='left')

In [141]:
trx_w_grp_prom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357772 entries, 0 to 357771
Data columns (total 22 columns):
store                    357772 non-null object
sku                      357772 non-null int64
sale1                    357772 non-null int64
sale2                    357772 non-null int64
sale3                    357772 non-null int64
return                   357772 non-null int64
date                     357772 non-null datetime64[ns]
STORE_ALT                357772 non-null object
SKU_ALT                  357772 non-null object
KEY                      357772 non-null object
CAMP_FLYER_FLYER         357772 non-null int64
CAMP_MONEY_ERP           357772 non-null int64
CAMP_NONE_NONE           357772 non-null int64
CAMP_OTHER_DEPTEXTR      357772 non-null int64
CAMP_SPECIAL_ARRIVAL     357772 non-null int64
CAMP_SPECIAL_DONTMISS    357772 non-null int64
CAMP_SPECIAL_INDIR       357772 non-null int64
CAMP_SPECIAL_MEDIA       357772 non-null int64
CAMP_SPECIAL_MEDIA2      35777

In [142]:
camp_skus=trx_prom.drop_duplicates(subset=['SKU_ALT_x','GroupPrm'])
camp_skus=camp_skus[['SKU_ALT_x','GroupPrm']]
camp_skus['SCORE']=1


In [143]:
camp_skus_grp=camp_skus.pivot_table(values=['SCORE'],index=['SKU_ALT_x'],columns=['GroupPrm'],aggfunc=lambda x: max(x),fill_value=0)

In [144]:
camp_skus_grp_table = pd.DataFrame(camp_skus_grp.to_records())
camp_skus_grp_table.columns = [hdr.replace("('SCORE', '", "MEM_").replace("')", "") 
                              for hdr in camp_skus_grp_table.columns]
camp_skus_grp_table.rename(columns={'SKU_ALT_x': 'SKU_ALT'}, inplace=True)

In [145]:
trx_w_grp_prom_mem=pd.merge(trx_w_grp_prom,camp_skus_grp_table,on=['SKU_ALT'],how='left')


In [146]:
trx_w_grp_prom_mem.head()

,store,sku,sale1,sale2,sale3,return,date,STORE_ALT,SKU_ALT,KEY,...,MEM_NONE_NONE,MEM_OTHER_DEPTEXTR,MEM_SPECIAL_ARRIVAL,MEM_SPECIAL_DONTMISS,MEM_SPECIAL_INDIR,MEM_SPECIAL_MEDIA,MEM_SPECIAL_MEDIA2,MEM_SPECIAL_MTHRSDY,MEM_STORE_PROMOPROD,MEM_STORE_VACOUTDOOR
0,STORE4,1084123,0,0,0,-6,2012-04-01,GENERAL,1084123,GENERAL1084123,...,1,0,0,0,0,0,0,0,0,0
1,STORE12,1116257,0,0,0,-1,2012-04-01,STORE12,1116257,STORE121116257,...,1,0,0,0,0,0,0,0,0,0
2,STORE7,1116407,0,0,0,-1,2012-04-01,GENERAL,1116407,GENERAL1116407,...,1,0,0,0,0,0,0,1,0,0
3,STORE10,1117146,0,1,0,0,2012-04-01,GENERAL,1117146,GENERAL1117146,...,1,0,0,0,0,0,0,0,0,0
4,STORE7,1117427,0,0,0,-1,2012-04-01,GENERAL,1117427,GENERAL1117427,...,1,0,1,0,1,0,0,0,0,0


In [147]:
trx_w_grp_prom_mem['TOT_SALES']=trx_w_grp_prom_mem[['sale1','sale2','sale3']].apply(lambda x: sum(x),axis=1)
trx_w_grp_prom_mem['NET_SALES']=trx_w_grp_prom_mem[['TOT_SALES','return']].apply(lambda x: sum(x),axis=1)

from scipy import stats


In [148]:

#dataraw=trx_w_grp_prom_mem.loc[trx_w_grp_prom_mem['MEM_FLYER_FLYER']==1]


#print('MEM_FLYER_FLYER'+'Results')

#t-test
#print( stats.ttest_ind(dataraw['CAMP_FLYER_FLYER'],dataraw['NET_SALES'])  )


#descriptive statistics
#print(  dataraw[['CAMP_FLYER_FLYER','NET_SALES']].groupby('CAMP_FLYER_FLYER').describe(percentiles=[0.01,0.05,0.10,0.25,
#                                                                                            0.50,0.75,0.90,0.95,.99])  )


#decision_tree

#feature_cols=['CAMP_STORE_VACOUTDOOR','CAMP_STORE_PROMOPROD','CAMP_SPECIAL_MTHRSDY',
#                   'CAMP_SPECIAL_MEDIA2','CAMP_SPECIAL_MEDIA','CAMP_SPECIAL_INDIR','CAMP_SPECIAL_DONTMISS',
#                   'CAMP_SPECIAL_ARRIVAL','CAMP_OTHER_DEPTEXTR','CAMP_MONEY_ERP','CAMP_FLYER_FLYER']

#model = ExtraTreesClassifier()
#model.fit(dataraw[feature_cols],dataraw['NET_SALES'])
#print(model.feature_importances_) 
#feat_importances = pd.Series(model.feature_importances_, index=dataraw[feature_cols].columns)

#print(feat_importances.sort_values(ascending=False))

#dtree=DecisionTreeRegressor()
#dtree.fit(dataraw[feature_cols],dataraw['NET_SALES'])
#dot_data = StringIO()
#export_graphviz(dtree, out_file=dot_data,  
#                filled=True, rounded=True,impurity=True,
#                special_characters=True, feature_names = feature_cols, node_ids=True, proportion=True)
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())


In [161]:



def CompareResults(X):
    
    dataraw=trx_w_grp_prom_mem.loc[trx_w_grp_prom_mem['MEM_'+X]==1]


    print('MEM_'+X+'Results')

    #t-test
    print( stats.ttest_ind(dataraw['CAMP_'+X],dataraw['NET_SALES'])  )


    #descriptive statistics
    print(  dataraw[['CAMP_'+X,'NET_SALES']].groupby('CAMP_'+X).describe(percentiles=[0.01,0.05,0.10,0.25,
                                                                                                0.50,0.75,0.90,0.95,.99])  )


    #decision_tree
    feature_cols=['CAMP_STORE_VACOUTDOOR','CAMP_STORE_PROMOPROD','CAMP_SPECIAL_MTHRSDY',
                       'CAMP_SPECIAL_MEDIA2','CAMP_SPECIAL_MEDIA','CAMP_SPECIAL_INDIR','CAMP_SPECIAL_DONTMISS',
                       'CAMP_SPECIAL_ARRIVAL','CAMP_OTHER_DEPTEXTR','CAMP_MONEY_ERP','CAMP_FLYER_FLYER']

    np.random.seed(1234)

    model = ExtraTreesClassifier()
    model.fit(dataraw[feature_cols],dataraw['NET_SALES'])
    print(model.feature_importances_) 
    feat_importances = pd.Series(model.feature_importances_, index=dataraw[feature_cols].columns)

    print(feat_importances.sort_values(ascending=False))

    dtree=DecisionTreeRegressor()
    dtree.fit(dataraw[feature_cols],dataraw['NET_SALES'])
    dot_data = StringIO()
    export_graphviz(dtree, out_file=dot_data,  
                    filled=True, rounded=True,impurity=True,
                    special_characters=True, feature_names = feature_cols, node_ids=True, proportion=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    Image(graph.create_png())

In [164]:
#print(trx_w_grp_prom_mem['CAMP_FLYER_FLYER'].unique())         
#print(trx_w_grp_prom_mem['CAMP_MONEY_ERP'].unique())           
#print(trx_w_grp_prom_mem['CAMP_NONE_NONE'].unique())           
#print(trx_w_grp_prom_mem['CAMP_OTHER_DEPTEXTR'].unique())      
#print(trx_w_grp_prom_mem['CAMP_SPECIAL_ARRIVAL'].unique())     
#print(trx_w_grp_prom_mem['CAMP_SPECIAL_DONTMISS'].unique())    
#print(trx_w_grp_prom_mem['CAMP_SPECIAL_INDIR'].unique())       
#print(trx_w_grp_prom_mem['CAMP_SPECIAL_MEDIA'].unique())       
#print(trx_w_grp_prom_mem['CAMP_SPECIAL_MEDIA2'].unique())      
#print(trx_w_grp_prom_mem['CAMP_SPECIAL_MTHRSDY'].unique())     
#print(trx_w_grp_prom_mem['CAMP_STORE_PROMOPROD'].unique())     
#print(trx_w_grp_prom_mem['CAMP_STORE_VACOUTDOOR'].unique())    
#print(trx_w_grp_prom_mem['MEM_FLYER_FLYER'].unique())          
#print(trx_w_grp_prom_mem['MEM_MONEY_ERP'].unique())            
#print(trx_w_grp_prom_mem['MEM_NONE_NONE'].unique())            
#print(trx_w_grp_prom_mem['MEM_OTHER_DEPTEXTR'].unique())       
#print(trx_w_grp_prom_mem['MEM_SPECIAL_ARRIVAL'].unique())      
#print(trx_w_grp_prom_mem['MEM_SPECIAL_DONTMISS'].unique())     
#print(trx_w_grp_prom_mem['MEM_SPECIAL_INDIR'].unique())        
#print(trx_w_grp_prom_mem['MEM_SPECIAL_MEDIA'].unique())        
#print(trx_w_grp_prom_mem['MEM_SPECIAL_MEDIA2'].unique())       
#print(trx_w_grp_prom_mem['MEM_SPECIAL_MTHRSDY'].unique())      
#print(trx_w_grp_prom_mem['MEM_STORE_PROMOPROD'].unique())      
#print(trx_w_grp_prom_mem['MEM_STORE_VACOUTDOOR'].unique())  
list=['STORE_VACOUTDOOR','STORE_PROMOPROD','SPECIAL_MTHRSDY',
                       'SPECIAL_MEDIA2','SPECIAL_MEDIA','SPECIAL_INDIR','SPECIAL_DONTMISS',
                       'SPECIAL_ARRIVAL','OTHER_DEPTEXTR','MONEY_ERP','FLYER_FLYER']
for i in range(len(list)):
    CompareResults(list[i])

MEM_STORE_VACOUTDOORResults
Ttest_indResult(statistic=-38.688003835759304, pvalue=0.0)
                      NET_SALES                                                \
                          count      mean       std  min    1%   5%  10%  25%   
CAMP_STORE_VACOUTDOOR                                                           
0                        9814.0  2.966171  7.601246 -6.0 -1.00  1.0  1.0  1.0   
1                          73.0  1.438356  1.247067 -2.0 -0.56  1.0  1.0  1.0   

                                                         
                       50%  75%  90%  95%    99%    max  
CAMP_STORE_VACOUTDOOR                                    
0                      1.0  2.0  5.0  9.0  29.00  224.0  
1                      1.0  1.0  2.8  3.0   6.28    7.0  
[ 0.02069644  0.          0.          0.36320157  0.00834695  0.          0.
  0.01783455  0.1486918   0.          0.44122869]
CAMP_FLYER_FLYER         0.441229
CAMP_SPECIAL_MEDIA2      0.363202
CAMP_OTHER_DEPTEXTR   

MEM_SPECIAL_ARRIVALResults
Ttest_indResult(statistic=-122.32022453466308, pvalue=0.0)
                     NET_SALES                                                \
                         count      mean       std   min   1%   5%  10%  25%   
CAMP_SPECIAL_ARRIVAL                                                           
0                      33669.0  5.964537  9.041710 -44.0 -1.0  1.0  1.0  1.0   
1                       2017.0  3.478929  5.681238 -16.0 -1.0  1.0  1.0  1.0   

                                                          
                      50%  75%   90%   95%    99%    max  
CAMP_SPECIAL_ARRIVAL                                      
0                     2.0  7.0  16.0  23.0  41.00  205.0  
1                     2.0  3.0   8.0  14.0  28.84   74.0  
[ 0.01353261  0.04799181  0.          0.10394339  0.07799646  0.10537081
  0.15219408  0.22975241  0.          0.10276859  0.16644985]
CAMP_SPECIAL_ARRIVAL     0.229752
CAMP_FLYER_FLYER         0.166450
CAMP_SPECIAL_DO

In [165]:
np.random.seed(1234)

CompareResults('STORE_VACOUTDOOR')

MEM_STORE_VACOUTDOORResults
Ttest_indResult(statistic=-38.688003835759304, pvalue=0.0)
                      NET_SALES                                                \
                          count      mean       std  min    1%   5%  10%  25%   
CAMP_STORE_VACOUTDOOR                                                           
0                        9814.0  2.966171  7.601246 -6.0 -1.00  1.0  1.0  1.0   
1                          73.0  1.438356  1.247067 -2.0 -0.56  1.0  1.0  1.0   

                                                         
                       50%  75%  90%  95%    99%    max  
CAMP_STORE_VACOUTDOOR                                    
0                      1.0  2.0  5.0  9.0  29.00  224.0  
1                      1.0  1.0  2.8  3.0   6.28    7.0  
[ 0.02069644  0.          0.          0.36320157  0.00834695  0.          0.
  0.01783455  0.1486918   0.          0.44122869]
CAMP_FLYER_FLYER         0.441229
CAMP_SPECIAL_MEDIA2      0.363202
CAMP_OTHER_DEPTEXTR   

In [168]:
promotion.head()

,dateFrom,dateThru,Group,promotionType,sku,store
0,1.01.2012,31.01.2012,Special,Media,1122735,NaN
1,1.01.2012,31.01.2012,Special,Media,1114952,NaN
2,1.01.2012,31.01.2012,Special,Media,1100480,NaN
3,6.01.2012,2.02.2012,flyer,Flyer,2014630,NaN
4,6.01.2012,2.02.2012,flyer,Flyer,1111699,NaN


In [169]:
promotion['store']=promotion['store'].fillna('GENERAL')
promotion.head()

,dateFrom,dateThru,Group,promotionType,sku,store
0,1.01.2012,31.01.2012,Special,Media,1122735,GENERAL
1,1.01.2012,31.01.2012,Special,Media,1114952,GENERAL
2,1.01.2012,31.01.2012,Special,Media,1100480,GENERAL
3,6.01.2012,2.02.2012,flyer,Flyer,2014630,GENERAL
4,6.01.2012,2.02.2012,flyer,Flyer,1111699,GENERAL


In [179]:
y=promotion.loc[promotion['store'].isin(['STORE9','STORE12'])]

In [181]:
y['store'].unique()

array(['STORE12', 'STORE9'], dtype=object)